First, setup parameters about insight we want to work on. 

Now prepare insight analyzer instance.

In [ ]:
from insight_analyzer import *

analyzer = ClusterAnalyzer()

We are ready now. Let's get to work.
First fetch the dataframe from the server.

In [ ]:
df = analyzer.get_df()

Let's see the data:

In [ ]:
df.head(5)

Awesome! Now let's see them in a plot!

In [ ]:
plt.scatter(
    df[df.columns[0]],
    df[df.columns[1]]
)
plt.show()

Ok, looks good. Let's set up our predictor!

In [ ]:
# Threshold for cluster proximity, lower promotes splitting
threshold = 0.03

cluster_count = 3

# We need to update our DF to be compatible with Birch
x = np.column_stack((df[df.columns[0]], df[df.columns[1]]))

model = Birch(threshold=threshold, n_clusters=cluster_count)
yhat = model.fit_predict(x)


Now let's preview the clusters!

In [ ]:
for cluster in np.unique(yhat):
    plt.scatter(df.values[yhat == cluster, 0], df.values[yhat == cluster, 1])
plt.show()

Altough this looks promising, we will get better results, if we normalize the coordinates. To do so, let's use the `MinMaxScaler`.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the data
scaler = MinMaxScaler(feature_range=(0,1))
normalized_df = scaler.fit_transform(df)
normalized_df = pd.DataFrame(normalized_df, columns=df.columns)


Ok, we can run our predictor again, with the same setting as before, so we do not change too much at a time.

In [ ]:
from sklearn.cluster import Birch

# Threshold for cluster proximity, lower promotes splitting
threshold = 0.03

cluster_count = 3

# We need to update our DF to be compatible with Birch
x = np.column_stack(
    (normalized_df[normalized_df.columns[0]],
     normalized_df[normalized_df.columns[1]])
)

model = Birch(threshold=threshold, n_clusters=cluster_count)
yhat = model.fit_predict(x)

One last preview, before pushing the data to server.

In [ ]:
for cluster in np.unique(yhat):
    plt.scatter(df.values[yhat == cluster, 0], df.values[yhat == cluster, 1])
plt.show()

Awesome! Last thing to do now is to push it to the server, so we can see it on our dashboard.

In [ ]:
analyzer.push_to_server(yhat)